In [1]:
import pandas as pd
import numpy as np
import math


In [3]:
data= pd.read_csv('/content/DecisionTreeDataset - Sheet1 (2).csv')

In [4]:
data

,Day,Temperature,Outlook,Humidity,Windy,Play Golf?
0,07-05,hot,sunny,high,False,no
1,07-06,hot,sunny,high,True,no
2,07-07,hot,overcast,high,False,yes
3,07-09,cool,rain,normal,False,yes
4,07-10,cool,overcast,normal,True,yes
5,07-12,mild,sunny,high,False,no
6,07-14,cool,sunny,normal,False,yes
7,07-15,mild,rain,normal,False,yes
8,07-20,mild,sunny,normal,True,yes
9,07-21,mild,overcast,high,True,yes


In [5]:

df = pd.DataFrame(data)
def entropy(data):
    # entropy
    labels = data['Play Golf?']
    total_instances = len(labels)
    unique_labels = labels.unique()
    entropy_val = 0

    for label in unique_labels:
        p = len(labels[labels == label]) / total_instances
        entropy_val -= p * math.log2(p)

    return entropy_val

def information_gain(data, attribute):
    #information gain
    total_instances = len(data)
    attribute_entropy = 0

    for value in data[attribute].unique():
        subset = data[data[attribute] == value]
        subset_entropy = entropy(subset) * len(subset) / total_instances
        attribute_entropy += subset_entropy

    return entropy(data) - attribute_entropy

def build_decision_tree(data, attributes):
    # if all instances have the same class, return the class
    if len(data['Play Golf?'].unique()) == 1:
        return data['Play Golf?'].iloc[0]

    # if no attributes left to split, return the majority class
    if len(attributes) == 0:
        return data['Play Golf?'].value_counts().idxmax()

    # highest information gain
    max_gain = -1
    best_attribute = None
    for attribute in attributes:
        gain = information_gain(data, attribute)
        if gain > max_gain:
            max_gain = gain
            best_attribute = attribute

    # Create a subtree
    tree = {best_attribute: {}}
    remaining_attributes = [attr for attr in attributes if attr != best_attribute]

    for value in data[best_attribute].unique():
        subset = data[data[best_attribute] == value]
        subtree = build_decision_tree(subset, remaining_attributes)
        tree[best_attribute][value] = subtree

    return tree


attributes = ['Outlook', 'Temperature', 'Humidity', 'Windy']


decision_tree = build_decision_tree(df, attributes)


print(decision_tree)

{'Outlook': {'sunny': {'Humidity': {'high': 'no', 'normal': 'yes'}}, 'overcast': 'yes', 'rain': {'Windy': {False: 'yes', True: 'no'}}}}


In [ ]:
testing_data = {
    'Day': ['Today', 'Tomorrow'],
    'Temperature': ['cool', 'mild'],
    'Outlook': ['sunny', 'sunny'],
    'Humidity': ['normal', 'normal'],
    'Windy': [False, False]
}

In [ ]:
testing_df = pd.DataFrame(testing_data)

def predict(instance, tree):
    # Recursively traverse the decision tree to make a prediction
    attribute = next(iter(tree))
    value = instance[attribute]
    subtree = tree[attribute][value]

    if isinstance(subtree, dict):
        return predict(instance, subtree)
    else:
        return subtree


In [ ]:
# Make predictions for the testing data
predictions = []
for i in range(len(testing_df)):
    instance = testing_df.iloc[i]
    prediction = predict(instance, decision_tree)
    predictions.append(prediction)

# Add predictions to the testing data DataFrame
testing_df['Play Golf?'] = predictions

# Print the final predictions for the testing data
print(testing_df[['Day', 'Play Golf?']])